# Tarea 3 - Optimización
- Integrantes: 
    - Samuel Coña J. - 2020451320
    - Daniel Lopez B. -
    - Dayan Sáez C. - 2020444854
    - Luis Valenzuela C. - 2020437548

Instancia a resolver:

<table>
  <tr>
    <th></th>
    <th colspan="4">
      Máquinas r</th>
  </tr>
  <tr>
    <th>Trabajo i</th>
    <th>M<sub>1</th>
    <th>M<sub>2</th>
    <th>M<sub>3</th>
    <th>M<sub>4</th>
  </tr>
  <tr>
    <th>J<sub>1</th>
    <th>9</th>
    <th>13</th>
    <th>15</th>
    <th>20</th>
  </tr>
  <tr>
    <th>J<sub>2</th>
    <th>11</th>
    <th>17</th>
    <th>18</th>
    <th>24</th>
  </tr>
  <tr>
    <th>J<sub>3</th>
    <th>8</th>
    <th>12</th>
    <th>14</th>
    <th>18</th>
  </tr>
  <tr>
    <th>J<sub>4</th>
    <th>6</th>
    <th>10</th>
    <th>12</th>
    <th>15</th>
  </tr>

In [18]:
T = [[9,13,15,20],
     [11,17,18,24],
     [8,12,14,18],
     [6,10,12,15]]

### Párametros de los modelos
$ T_{ri} \text{: Duración de proceso del trabajo i en maquina r.} \\$
$ B_{rj} \text{: Tiempo de inicio del trabajo j, en la posición de la secuencia, en máquina r.} \\$
$ C_{ri} \text{: Tiempo de finalización del trabajo i en la máquina r} \\$
$ D_{ki} \text{: 1 si el } \\$
$ E_{rj} \text{: Hola} \\$
$ q_{rik} \text{: Hola} \\$
$ S_{ri} \text{: Hola} \\$
$ X_{rj} \text{: Hola} \\$
$ Y_{rj} \text{: Hola} \\$
$ Z_{rj} \text{: Hola} \\$

### Modelo Wagner

$$ Min \quad C_{max} = C_{MN} $$
$$ \sum_{j=1}^{N} Z_{ij} = 1; \quad (1 \le i \le N), \quad (1) $$ 
$$ \sum_{i=1}^{N} Z_{ij} = 1; \quad (1 \le j \le N), \quad (2)$$
$$ \sum_{i=1}^{N} T_{ri}Z_{i,j+1} - \sum_{i=1}^{N} T_{r+1,i}Z_{ij} + X_{r,j+1} - X_{r+1,j+1} + Y_{r,j+1} - Y_{rj} = 0; \quad (1 \le r \le M-1;1 \le j \le N-1), \quad (3)
$$

$$ \sum_{i=1}^{N} T_{ri}Z_{i1} + X_{r1} - X_{r+1,1} + Y_{r1} = 0; \quad (1 \le r \le M-1), \quad (4) $$

$$ C_{MN} = \sum_{i=1}^{N} T_{Mi} + \sum_{p=1}^{N} X_{Mp}. \quad (5) $$

#### Implementación en PuLP

#### Implementación en CPLEX

#### Implementación en GUROBI

### Modelo Wilson
$$ Min \quad C_{max} = B_{MN} + \sum_{i=1}^{N}T_{Mi}Z_{iN} $$
$$ \sum_{j=1}^{N} Z_{ij} = 1; \quad (1 \le i \le N), \quad (1) $$ 
$$ \sum_{i=1}^{N} Z_{ij} = 1; \quad (1 \le j \le N), \quad (2)$$
$$ B_{1j}+\sum_{i=1}^{N} T_{1i}Z_{1j} = B_{1,j+1}; \quad (1\leq j \leq N-1), \quad (3)$$
$$ B_{11} = 0, \quad(4)$$
$$ B_{r1} + \sum_{i=1}^{N} T_{ri}Z_{i1} = B_{r+1,1}; \quad (1\leq r \leq M-1),\quad (5) $$

$$ B_{rj} + \sum_{i=1}^{N}T_{ri}Z_{ij} \leq B_{r+1,j}; \quad (1\leq r\leq M-1;2\leq j \leq N), \quad (6) $$
$$ B_{rj} + \sum_{i=1}^N T_{ri}Z_{ij} \leq B_{r,j+1}; \quad (2\leq r\leq M; 1\leq j\leq N-1). \quad (7) $$

#### Implementación en PuLP

#### Implementación en CPLEX

#### Implementacion en GUROBI

### Modelo Manne
$$ Min \quad C_{max}$$
$$ subject \quad to$$
$$ C_{1i} >= T_{1i}; \quad (i<= i <= N), \quad (1)$$
$$ C_{ri} - C_{r-1,i} >= T_{ri}; \quad (2 <= r <= M; 1 <= i <= N), \quad (2)$$
$$ C_{ri} - C_{rk} + PD_{ik} >= T_{ri}; \quad (1 <= r <= M; 1 <= i < k <= N), \quad (3)$$
$$ C_{ri} - C{rk} + PD_{ik} <= P - T_{rk}; \quad (1 <= r <= M; 1 <= i < k <= N) \quad (4)$$
$$ C_{max} >= C_{Mi}; \quad (1 <= i <= N) \quad (5)$$

#### Implementación de PuLP

#### Implementación con CPLEX

#### Implementación con GUROBI

### Modelo Liao-You

#### Implementación en PuLP

#### Implementación en CPLEX

#### Implementación en GUROBI